In [ ]:
# ---- EI139 PS1 - Problem 2 (R) ----
# Reproducible setup
set.seed(2025)

log_marglik <- function(y, X) {
  n <- nrow(X); K <- ncol(X)
  yty <- sum(y*y)
  out <- numeric(K)
  for (k in 1:K) {
    Xk <- X[, 1:k, drop=FALSE]
    Sk <- diag(k) + crossprod(Xk)           # I_k + X'X
    L  <- chol(Sk)
    logdet <- 2*sum(log(diag(L)))
    v <- forwardsolve(t(L), crossprod(Xk, y))  # L z = X' y
    w <- backsolve(L, v)                        # L' w = z => w = S^{-1} X' y
    quad <- sum((Xk %*% w) * y)                 # y' X S^{-1} X' y
    out[k] <- -0.5*n*log(2*pi) - 0.5*(yty - quad) - 0.5*logdet
  }
  out
}

run_once <- function(n, kmax=10, k0=4, M=1000) {
  Z <- matrix(rnorm(n*kmax), n, kmax)
  qrZ <- qr(Z)
  X <- qr.Q(qrZ)                 # thin Q with orthonormal columns
  beta <- c(rep(0.5, k0), rep(0, kmax-k0))
  mu <- as.vector(X %*% beta)
  counts <- integer(kmax)
  post_sum <- numeric(kmax)
  for (m in 1:M) {
    y <- mu + rnorm(n)
    lml <- log_marglik(y, X)
    w <- exp(lml - max(lml))
    post <- w / sum(w)
    kstar <- which.max(post)
    counts[kstar] <- counts[kstar] + 1
    post_sum <- post_sum + post
  }
  data.frame(k=1:kmax,
             freq_selected=counts,
             avg_posterior=post_sum/M,
             freq_selected_share=counts/M)
}

res50  <- run_once(50,  kmax=10, k0=4, M=1000)
res100 <- run_once(100, kmax=10, k0=4, M=1000)
res500 <- run_once(500, kmax=10, k0=4, M=1000)

# print(res50); print(res100); print(res500)
# library(xtable)

# res50$N  <- 50
# res100$N <- 100
# res500$N <- 500

# res_all <- rbind(res50, res100, res500)
# print(xtable(res_all, caption = "Simulation results"), include.rownames = FALSE, file = "PS1_results.tex")
if (!requireNamespace("tidyverse", quietly = TRUE)) install.packages("tidyverse")
if (!requireNamespace("knitr", quietly = TRUE)) install.packages("knitr")

library(tidyverse)
library(knitr)

res50$N  <- 50
res100$N <- 100
res500$N <- 500
res_all <- bind_rows(res50, res100, res500)

# 1) Average posterior probability 
avg_table <- res_all %>%
  select(k, N, avg_posterior) %>%
  pivot_wider(names_from = N, names_prefix = "N_", values_from = avg_posterior) %>%
  arrange(k)

avg_latex <- knitr::kable(
  avg_table,
  format = "latex",
  booktabs = TRUE,
  caption = "Average posterior probability by $k$ and sample size",
  digits = 3,
  label = "tab:avg_posterior"
)

# 2) Selection frequency (share)
freq_table <- res_all %>%
  select(k, N, freq_selected_share) %>%
  pivot_wider(names_from = N, names_prefix = "N_", values_from = freq_selected_share) %>%
  arrange(k)

freq_latex <- knitr::kable(
  freq_table,
  format = "latex",
  booktabs = TRUE,
  caption = "Selection frequency (share) by $k$ and sample size",
  digits = 3,
  label = "tab:freq_selected"
)

writeLines(as.character(avg_latex), con = "PS1_avg_posterior.tex")
writeLines(as.character(freq_latex), con = "PS1_freq_selected.tex")

Warning message:
"package 'purrr' was built under R version 4.5.1"
── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.2     ✔ tibble    3.3.0
✔ lubridate 1.9.4     ✔ tidyr     1.3.1
✔ purrr     1.1.0     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


ERROR: [1m[33mError[39m in `select()`:[22m
[33m![39m Can't select columns that don't exist.
[31m✖[39m Column `N` doesn't exist.


In [ ]:
# ---- EI139 PS1 - Problem 2 (Julia) ----
using Random, LinearAlgebra, Statistics, Printf

Random.seed!(2025)

function log_marglik(y::Vector{Float64}, X::Matrix{Float64})
    n, K = size(X)
    yty = dot(y, y)
    out = Vector{Float64}(undef, K)
    for k in 1:K
        Xk = @view X[:, 1:k]
        Sk = I + Xk' * Xk
        L = cholesky(Sk).L
        logdet = 2sum(log.(diag(L)))
        v = L \ (Xk' * y)      # solve L * v = X' y
        w = L' \ v             # solve L' * w = v
        quad = dot(Xk * w, y)  # y' X S^{-1} X' y
        out[k] = -0.5*n*log(2π) - 0.5*(yty - quad) - 0.5*logdet
    end
    return out
end

function run_once(n::Int; kmax::Int=10, k0::Int=4, M::Int=1000)
    Z = randn(n, kmax)
    Q, R = qr(Z)
    X = Matrix(Q)                  # thin-Q
    beta = vcat(fill(0.5, k0), zeros(kmax - k0))
    mu = X * beta
    counts = zeros(Int, kmax)
    post_sum = zeros(kmax)
    for m in 1:M
        y = mu .+ randn(n)
        lml = log_marglik(y, X)
        w = exp.(lml .- maximum(lml))
        post = w ./ sum(w)
        kstar = argmax(post)
        counts[kstar] += 1
        post_sum .+= post
    end
    return (; k=collect(1:kmax),
            freq_selected=counts,
            avg_posterior=post_sum ./ M,
            freq_selected_share=counts ./ M)
end

res50  = run_once(50,  kmax=10, k0=4, M=1000)
res100 = run_once(100, kmax=10, k0=4, M=1000)
res500 = run_once(500, kmax=10, k0=4, M=1000)

println(res50); println(res100); println(res500)


(k = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], freq_selected = [397, 152, 112, 100, 42, 50, 35, 33, 47, 32], avg_posterior = [0.13723331702855954, 0.12863230433142495, 0.12064884222835356, 0.11336896579192918, 0.10175808149012087, 0.09169964362045664, 0.08417106282071664, 0.07771202212792949, 0.0753841108294889, 0.06939164973102031], freq_selected_share = [0.397, 0.152, 0.112, 0.1, 0.042, 0.05, 0.035, 0.033, 0.047, 0.032])
(k = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], freq_selected = [435, 124, 94, 106, 44, 54, 30, 29, 39, 45], avg_posterior = [0.14321153997093078, 0.12998310091934087, 0.11964211317386182, 0.11287750893783965, 0.10017588358346732, 0.09174092169879598, 0.08225537146005756, 0.07704850146517125, 0.07229771256305643, 0.07076734622747818], freq_selected_share = [0.435, 0.124, 0.094, 0.106, 0.044, 0.054, 0.03, 0.029, 0.039, 0.045])
(k = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], freq_selected = [409, 129, 111, 84, 60, 39, 36, 29, 47, 56], avg_posterior = [0.13904531969399597, 0.1273701905632862, 0.118